**요약**
- 예측 결과 csv 파일의 작은 객체 (poles, traffic light, people)를 OneFormer 추론 결과로 대체합니다.

<br>

**Inputs:**
- `dir_save`: 예측 결과들이 저장될 디렉토리
- `fname`: 강화할 예측 결과물 (submission 파일)

<br>

**Outputs**:
- f`{dir_save}/result_4.pickle: 각 테스트 이미지에 대한 poles 객체 대체 결과 마스크
- f`{dir_save}/result_5.pickle: 각 테스트 이미지에 대한 traffic light 객체 대체 결과 마스크
- f`{dir_save}/result_9.pickle: 각 테스트 이미지에 대한 people 객체 대체 결과 마스크

In [1]:
dir_save = '../save'
fname = 'AverageEnsemble_Raw.csv'

# 소형 object class dectection 강화하기 (class 4:pole, 5: traffic light, 9: person)

## ade, coco, cityscape 데이터 셋을 우리 문제의 class 분류에 적절히 대응시켜 활용하기

In [14]:
# ade, coco, cityscape 데이터 셋을 우리 문제의 class 분류에 적절히 대응

ade2our = {
    0: 2,#"wall",
    1: 2,#"building",
    2: 8,#"sky",
    3: None,# "floor",
    4: 7,#"tree",
    5: 2,# "ceiling",
    6: 0,# "road, route",
    7: None,# "bed",
    8: None,# "window ",
    9: 7,# "grass",
    10: None,# "cabinet"
    11: 1,# "sidewalk, pavement",
    12: 9,# "person",
    13: 7,# "earth, ground",
    14: None,# "door",
    15: None,# "table",
    16: 7,# "mountain, mount",
    17: 7,# "plant",
    18: None,# "curtain",
    19: None,# "chair",
    20: 11,# "car"
    21: None,# "water",
    22: None,# "painting, picture",
    23: None,# "sofa",
    24: None,# "shelf",
    25: 2,# "house",
    26: None,#"sea",
    27: None,#"mirror",
    28: None,# "rug",
    29: 7,#"field",
    30: None,#"armchair",
    31: None,# "seat",
    32: 3,# "fence",
    33: None,# "desk",
    34: None,# "rock, stone",
    35: None,#"wardrobe, closet, press",
    36: None,# "lamp",
    37: None,#"tub"
    38: 2,#"rail"
    39: None,#"cushion",
    40: 2,#"base, pedestal, stand",
    41: None,# "box",
    42: 2,# "column, pillar",
    43: 6,#"signboard, sign",
    44: None,# "chest of drawers, chest, bureau, dresser",
    45: None,#"counter",
    46: None,#"sand",
    47: None,#"sink",
    48: 2,#"skyscraper",
    49: None,# "fireplace",
    50: None,#"refrigerator, icebox",
    51: None,#"grandstand, covered stand",
    52: None,#"path",
    53: 2,#"stairs",
    54: None,#"runway",
    55: None,#"case, display case, showcase, vitrine",
    56: None,#"pool table, billiard table, snooker table",
    57: None,#"pillow",
    58: None,#"screen door, screen",
    59: None,#"stairway, staircase","
    60: None,#"river",
    61: 2,#"bridge, span",
    62: None,#"bookcase",
    63: None,# "blind, screen",
    64: None,# "coffee table",
    65: None,#"toilet, can, commode, crapper, pot, potty, stool, throne",
    66: 7,#"flower",
    67: None,#"book",
    68: 7,# "hill",
    69: 2,# "bench",
    70: None,#"countertop",
    71: None,#"stove",
    72: 7,#"palm, palm tree",
    73: None,# "kitchen island",
    74: None,#"computer",
    75: None,#"swivel chair",
    76: None,# "boat",
    77: None,#"bar",
    78: None,#"arcade machine",
    79: None, #"hovel, hut, hutch, shack, shanty" 오두막집, 헛간
    80: 11, #"bus",
    81: None,# "towel",
    82: 2, #"light",
    83: 11,# "truck",
    84: 2, #"tower",
    85: None,# "chandelier",
    86: 2, #"awning, sunshade, sunblind",
    87: 4, #"street lamp",
    88: None,#  "booth",
    89: None,# "tv",
    90: None,# "plane",
    91: 2, #"dirt track",
    92: None, # "clothes",
    93: None, # "pole", ##################################################################
    94: None,# "land, ground, soil",
    95: 2, #"bannister, banister, balustrade, balusters, handrail",
    96: None,# "escalator, moving staircase, moving stairway",
    97: None,# "ottoman, pouf, pouffe, puff, hassock",
    98: None,# "bottle",
    99: None,# "buffet, counter, sideboard",
    100: 2, # "poster, posting, placard, notice, bill, card",
    101: None,# "stage",
    102: 11, #"van",
    103: None,# "ship",
    104: None,# "fountain",
    105: None,# "conveyer belt, conveyor belt, conveyer, conveyor, transporter",
    106: None,# "canopy",
    107: None,# "washer, automatic washer, washing machine",
    108: None,# "plaything, toy",
    109: None,# "pool",
    110: None,# "stool",
    111: None,# "barrel, cask",
    112: None,# "basket, handbasket",
    113: None,#  "falls",
    114: None,#  "tent",
    115: None,# "bag",
    116: 11, #"minibike, motorbike",
    117: None,# "cradle",
    118: None,# "oven",
    119: None,# "ball",
    120: None,# "food, solid food",
    121: 2, #"step, stair",
    122: None,# "tank, storage tank",
    123: 2, #"trade name",
    124: None,# "microwave",
    125: 2, #"pot",
    126: None,# "animal",
    127: None,# "bicycle",
    128: None,# "lake",
    129: None,# "dishwasher",
    130: None,# "screen",
    131: None,# "blanket, cover",
    132: 2, #"sculpture",
    133: None,# "hood, exhaust hood",
    134: None,# "sconce",
    135: None,# "vase",
    136: 5, #"traffic light",
    137: None,# "tray",
    138: 2, #"trash can",
    139: None,# "fan",
    140: None,# "pier",
    141: None,# "crt screen",
    142: None,# "plate",
    143: None,# "monitor",
    144: None,# "bulletin board",
    145: None,# "shower",
    146: None,# "radiator",
    147: None,# "glass, drinking glass",
    148: None,# "clock",
    149: None,# "flag"
}

#None 값 13으로 치환 (12: 아예 모르는 것, 13: detect했지만 모른척하는 것)
ade2our = {key: 13 if value is None else value for key, value in ade2our.items()}
ade2our[-1]=12

#######################################################################################
co2our = {
    0: 9,  # "person",
    1: None,  # "bicycle",
    2: 11,  # "car",
    3: 11,  # "motorcycle",
    4: None,  # "airplane",
    5: 11,  # "bus",
    6: None,  # "train",
    7: 11,  # "truck",
    8: 2,  # "boat",
    9: 5,  # "traffic light",
    10: None,  # "fire hydrant",
    11: 6,  # "stop sign",
    12: None,  # "parking meter",
    13: 2,  # "bench",
    14: None,  # "bird",
    15: None,  # "cat",
    16: None,  # "dog",
    17: None,  # "horse",
    18: None,  # "sheep",
    19: None,  # "cow",
    20: None,  # "elephant",
    21: None,  # "bear",
    22: None,  # "zebra",
    23: None,  # "giraffe",
    24: 9,  # "backpack" 사람과 한 묶음,
    25: 2,  # "umbrella",
    26: 9,  # "handbag" 사람과 한 묶음,
    27: None,  # "tie",
    28: None,  # "suitcase",
    29: None,  # "frisbee",
    30: None,  # "skis",
    31: None,  # "snowboard",
    32: None,  # "sports ball",
    33: None,  # "kite",
    34: None,  # "baseball bat",
    35: None,  # "baseball glove",
    36: None,  # "skateboard",
    37: None,  # "surfboard",
    38: None,  # "tennis racket",
    39: None,  # "bottle",
    40: None,  # "wine glass",
    41: None,  # "cup",
    42: None,  # "fork",
    43: None,  # "knife",
    44: None,  # "spoon",
    45: None,  # "bowl",
    46: None,  # "banana",
    47: None,  # "apple",
    48: None,  # "sandwich",
    49: None,  # "orange",
    50: None,  # "broccoli",
    51: None,  # "carrot",
    52: None,  # "hot dog",
    53: None,  # "pizza",
    54: None,  # "donut",
    55: None,  # "cake",
    56: 2,  # "chair",
    57: None,  # "couch",
    58: 7,  # "potted plant",
    59: None,  # "bed",
    60: None,  # "dining table",
    61: None,  # "toilet",
    62: 2,  # "tv",
    63: None,  # "laptop",
    64: None,  # "mouse",
    65: None,  # "remote",
    66: None,  # "keyboard",
    67: None,  # "cell phone",
    68: None,  # "microwave",
    69: None,  # "oven",
    70: None,  # "toaster",
    71: None,  # "sink",
    72: None,  # "refrigerator",
    73: None,  # "book",
    74: None,  # "clock",
    75: None,  # "vase",
    76: None,  # "scissors",
    77: None,  # "teddy bear",
    78: None,  # "hair drier",
    79: None,  # "toothbrush",
    80: 2,  # "banner", ################ 애매 2 or 6 / 애매할땐 일단 점수 비중이 낮은거 부터 하고, 나중에 투표할때는 핵심 으로 (지금은 2처리) 나중 투표때는 6처리 요망.
    81: None,  # "blanket",
    82: None,  # "bridge",
    83: None,  # "cardboard",
    84: None,  # "counter",
    85: None,  # "curtain",
    86: 2,  # "door-stuff",
    87: None,  # "floor-wood",
    88: 7,  # "flower",
    89: None,  # "fruit",
    90: 0,  # "gravel",##############애매 0 or 7
    91: 2,  # "house",
    92: 2,  # "light",
    93: 2,  # "mirror-stuff",
    94: None,  # "net",
    95: None,  # "pillow",
    96: 0,  # "platform",
    97: None,  # "playingfield",
    98: 2,  # "railroad",
    99: None,  # "river",
    100: 0,  # "road",
    101: 2,  # "roof",
    102: None,  # "sand",
    103: None,  # "sea",
    104: None,  # "shelf",
    105: None,  # "snow",
    106: 2,  # "stairs",
    107: None,  # "tent",
    108: None,  # "towel",
    109: 2,  # "wall-brick",
    110: 2,  # "wall-stone",
    111: 2,  # "wall-tile",
    112: 2,  # "wall-wood",
    113: None,  # "water-other",
    114: 2,  # "window-blind",
    115: 2,  # "window-other",
    116: 7,  # "tree-merged",
    117: 3,  # "fence-merged",
    118: 2,  # "ceiling-merged",
    119: 8,  # "sky-other-merged",
    120: None,  # "cabinet-merged",
    121: None,  # "table-merged",
    122: None,  # "floor-other-merged",
    123: 1,  # "pavement-merged",
    124: 7,  # "mountain-merged",
    125: 7,  # "grass-merged",
    126: 0,  # "dirt-merged", ############# 애매 0 or 7
    127: None,  # "paper-merged",
    128: None,  # "food-other-merged",
    129: 2,  # "building-other-merged",
    130: 7,  # "rock-merged",
    131: 2,  # "wall-other-merged",
    132: None,  # "rug-merged"
}

#None 값 13으로 치환 (12: 아예 모르는 것, 13: detect했지만 모른척하는 것)
co2our = {key: 13 if value is None else value for key, value in co2our.items()}
co2our[-1]=12

#######################################################################################
city2our = {
    0: 0,  # "road",
    1: 1,  # "sidewalk",
    2: 2,  # "building",
    3: 2,  # "wall",
    4: 3,  # "fence",
    5: 4,  # "pole",
    6: 5,  # "traffic light",
    7: 6,  # "traffic sign",
    8: 7,  # "vegetation",
    9: 7,  # "terrain",
    10: 8,  # "sky",
    11: 9,  # "person",
    12: 10,  # "rider",
    13: 11,  # "car",
    14: 11,  # "truck",
    15: 11,  # "bus",
    16: None,  # "train", ########## 2 vs 11 둘다 비등해서 UN ..
    17: 11,  # "motorcycle",
    18: None,  # "bicycle"   ######## UN ..
}
#None 값 13으로 치환 (12: 아예 모르는 것, 13: detect했지만 모른척하는 것)
city2our = {key: 13 if value is None else value for key, value in city2our.items()}
city2our[-1]=12

In [15]:
%cd ./OneFormer

import os
import pickle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import math

def ade_replace_class(info):
    # Create a mapping from id to category_id
    id_to_category_id = {entry['id']: entry['category_id'] for entry in info[1]}
    id_to_category_id[0]=-1 ## 0 -> -1 -> 12 / ' ' -> None -> 13
    seg_np = info[0]
    # Map id to category_id
    seg_category_id = np.vectorize(id_to_category_id.__getitem__)(seg_np)
    seg_final_class = np.vectorize(ade2our.get)(seg_category_id)
    return seg_final_class

def co_replace_class(info):
    # Create a mapping from id to category_id
    id_to_category_id = {entry['id']: entry['category_id'] for entry in info[1]}
    id_to_category_id[0]=-1 ## 0 -> -1 -> 12 / ' ' -> None -> 13
    seg_np = info[0]
    # Map id to category_id
    seg_category_id = np.vectorize(id_to_category_id.__getitem__)(seg_np)
    seg_final_class = np.vectorize(co2our.get)(seg_category_id)
    return seg_final_class

def city_replace_class(info):
    # Create a mapping from id to category_id
    id_to_category_id = {entry['id']: entry['category_id'] for entry in info[1]}
    id_to_category_id[0]=-1 ## 0 -> -1 -> 12 / ' ' -> None -> 13
    seg_np = info[0]
    # Map id to category_id
    seg_category_id = np.vectorize(id_to_category_id.__getitem__)(seg_np)
    seg_final_class = np.vectorize(city2our.get)(seg_category_id)
    return seg_final_class

/home/eunwoo/experiment/PSSC/Oneformer(instance)/content/OneFormer


# 분석 시작

- Oneformer pretrained model로 test data set 추론하는 파일 ' ' 참조

In [4]:
# 시간절약을 위해 미리 test_data set에 대해서 ade,coco,cityscape data set의 dinit/swin version에 대해 미리 다 추론하여 저장해놓은 pickle 파일 불러오기


directory = f'../{dir_save}/ade'  
ade_pickle_files = []
# 디렉토리 내의 모든 파일을 검색
for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.pickle'):  # .pickle 확장자를 가진 파일만 선택
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:  # 바이너리 읽기 모드로 파일 열기
            data = pickle.load(f)  # 파일 내용 불러오기
        ade_pickle_files.append(data)  # 불러온 데이터를 리스트에 추가

# pickle_files 리스트에는 모든 .pickle 파일들의 데이터가 들어있습니다.
print("Ade_Loaded", len(ade_pickle_files), "pickle files")

directory = f'../{dir_save}/co'  
co_pickle_files = []
# 디렉토리 내의 모든 파일을 검색
for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.pickle'):  # .pickle 확장자를 가진 파일만 선택
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:  # 바이너리 읽기 모드로 파일 열기
            data = pickle.load(f)  # 파일 내용 불러오기
        co_pickle_files.append(data)  # 불러온 데이터를 리스트에 추가

# pickle_files 리스트에는 모든 .pickle 파일들의 데이터가 들어있습니다.
print("Co_Loaded", len(co_pickle_files), "pickle files")

directory = f'../{dir_save}/city'   
city_pickle_files = []
# 디렉토리 내의 모든 파일을 검색
for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.pickle'):  # .pickle 확장자를 가진 파일만 선택
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:  # 바이너리 읽기 모드로 파일 열기
            data = pickle.load(f)  # 파일 내용 불러오기
        city_pickle_files.append(data)  # 불러온 데이터를 리스트에 추가

# pickle_files 리스트에는 모든 .pickle 파일들의 데이터가 들어있습니다.
print("City_Loaded", len(city_pickle_files), "pickle files")


100%|██████████| 4/4 [00:06<00:00,  1.55s/it]


Ade_Loaded 4 pickle files


100%|██████████| 4/4 [00:06<00:00,  1.52s/it]


Co_Loaded 4 pickle files


100%|██████████| 4/4 [00:05<00:00,  1.35s/it]

City_Loaded 4 pickle files


# Reinforce 4,5,9 class mask

In [22]:
fname_dong = os.path.join(dir_save, fname)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm


def rle2mask(mask_rle, label=1, shape=(540, 960)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = label
    return img.reshape(shape)


df = pd.read_csv(fname_dong)

row = 0
target_masks = [] #target_masks
for i in tqdm(range(1898)):
    mask = np.zeros((540, 960)) + 12
    for c in range(12):
        if not df.loc[row, 'mask_rle'] == '-1':
            m = rle2mask(df.loc[row, 'mask_rle'], label=1)
            mask[np.where(m == 1)] = c
        row +=1

    mask = mask.astype(np.int32)
    target_masks.append(mask)


100%|██████████| 1898/1898 [00:28<00:00, 67.18it/s]


In [54]:
def make_target_bin_mask(target_mask,cls_num):
    bin_mask=(target_mask == cls_num)

    return bin_mask


def make_reinforce_ensemble_mask(picture_ind, cls_num):
    if cls_num not in [4,5,9]:
        raise Exception("Class must be one of 4, 5, and 9.")

    if cls_num==5: # use 'co_din_pa_pa.pickle', 'co_din_se_pa.pickle'
        second=co_replace_class(co_pickle_files[0][picture_ind])==cls_num
        second+=co_replace_class(co_pickle_files[1][picture_ind])==cls_num
        total = second

    elif cls_num ==4: # use 'city_din_pa_pa.pickle', 'city_swin_se_pa.pickle', 'city_swin_pa_pa.pickle', 'city_din_se_pa.pickle'
        third=city_replace_class(city_pickle_files[0][picture_ind])==cls_num
        third+=city_replace_class(city_pickle_files[1][picture_ind])==cls_num
        third+=city_replace_class(city_pickle_files[2][picture_ind])==cls_num
        third+=city_replace_class(city_pickle_files[3][picture_ind])==cls_num    
        total = third

    elif cls_num ==9: # use 'co_din_pa_pa.pickle','co_swin_se_pa.pickle', 'co_swin_pa_pa.pickle
        second=co_replace_class(co_pickle_files[0][picture_ind])==cls_num
        # second+=co_replace_class(co_pickle_files[1][picture_ind])==cls_num
        second+=co_replace_class(co_pickle_files[2][picture_ind])==cls_num
        second+=co_replace_class(co_pickle_files[3][picture_ind])==cls_num    
        total = second
    # else:
    #     total= ade_replace_class(ade_pickle_files[0][picture_ind])==cls_num

    bin_mask_eunwoo = total.astype(np.uint8)
    bin_mask_eunwoo = cv2.resize(bin_mask_eunwoo, (960, 540), interpolation=cv2.INTER_NEAREST)

    if cls_num==4: #가로등 가로 부분 제거
        kernel = np.ones((10, 1), np.uint8) 
        bin_mask_eunwoo = cv2.erode(bin_mask_eunwoo, kernel, iterations=2)
        bin_mask_eunwoo = cv2.dilate(bin_mask_eunwoo, kernel, iterations=2)

    return bin_mask_eunwoo

def choose_component_A_on_B(A_mask,B_mask):
    # 연결된 컴포넌트 레이블링 수행
    num_labels, labels = cv2.connectedComponents(A_mask)

    # 원래의 street_lamp_mask에서 겹치는 연결된 컴포넌트 전체를 표시하는 마스크 초기화
    final_mask = np.zeros_like(A_mask)

    # 각 겹치는 영역에 대해
    for i in range(1, num_labels):  # 0은 배경이므로 제외
        # i 번째 레이블과 겹치는 street_lamp_mask의 연결된 컴포넌트를 찾기
        component_mask = (labels == i)
        overlap = np.logical_and(component_mask, B_mask)
        
        # 겹치는 부분이 있다면 component_mask 추가
        if np.any(overlap):
            final_mask = np.logical_or(final_mask, component_mask)

    return final_mask

def reinforce_459(target_mask,picture_ind,cls_num):
    bin_reinforce_mask  = make_reinforce_ensemble_mask(picture_ind, cls_num)
    bin_mask_target = make_target_bin_mask(target_mask,cls_num)
    final_mask=choose_component_A_on_B(bin_reinforce_mask,bin_mask_target)
    predict_mask = final_mask
    return predict_mask


# 4,5,9 class reinforce 결과물

In [70]:
result_4={}
result_5={}
result_9={}

for pic_ind in tqdm(range(len(df))):

    target_mask= target_masks[pic_ind]

    result_4[pic_ind] = reinforce_459(target_mask,pic_ind,cls_num=4)
    result_5[pic_ind] = reinforce_459(target_mask,pic_ind,cls_num=5)
    result_9[pic_ind] = reinforce_459(target_mask,pic_ind,cls_num=9)
    
    break # if you want to run this code, delete this part


100%|██████████| 2/2 [00:04<00:00,  2.42s/it]


In [243]:
import pickle

# save
with open(f'../{dir_save}/result_4.pickle', 'wb') as f:
    pickle.dump(result_4, f, pickle.HIGHEST_PROTOCOL)

# save
with open(f'../{dir_save}/result_5.pickle', 'wb') as f:
    pickle.dump(result_5, f, pickle.HIGHEST_PROTOCOL)

# save
with open(f'../{dir_save}/result_9.pickle', 'wb') as f:
    pickle.dump(result_9, f, pickle.HIGHEST_PROTOCOL)